## Coming up with some concrete example of what can be done linking DM and Camera (using Butler and eTraveler)

# Load all the appropriate libraries.
Load the EO access libraries and establish a connection to the eTraveler production database.

Repos available from:

https://github.com/lsst-camera-dh/eTraveler-clientAPI.git

https://github.com/lsst-camera-dh/datacat-utilities.git

In [1]:
from get_EO_analysis_results import get_EO_analysis_results
from get_EO_analysis_files import get_EO_analysis_files
from exploreFocalPlane import exploreFocalPlane
from exploreRaft import exploreRaft
from eTraveler.clientAPI.connection import Connection

These methods give three views into the Camera side eTraveler system:
 * `eR` describes the camera setup, based on how the devices are arranged in the raft.
 * `gf` provides an access point into the files generated on the camera side. I do not understand this at this point (2018-11-15).
 * `g` proves an access point into the camera validation tests that have been performed as part of a particular data run.

In [2]:
eR = exploreRaft(db='Prod')
gf = get_EO_analysis_files(db='Prod')
g = get_EO_analysis_results(db='Prod')

Load the DM stack, along with miscellaneous other libraries.  This is a direct copy from the DM Bootcam Fe55 notebook, because it's easier to copy.

In [3]:
%matplotlib inline

# system imports
from matplotlib import pylab as plt
from scipy.optimize import leastsq
import numpy
import os
import re

# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afwDisplay
from lsst.ip.isr import IsrTask
import lsst.afw.detection as afwDetection

Generate a Butler that is aware of the example bootcamp repository.  This is a TS8 repository with a single run, which I know ahead of time is `run=7086`.

In [4]:
REPO = '/project/bootcamp/repo_RTM-007/'
butler = Butler(REPO)

Once the Butler connection is made, it's helpful to learn what keys it expects to specify data.  `getKeys()` provides that list

In [5]:
butler.getKeys('raw')

{'run': str, 'visit': int, 'detectorName': str, 'detector': int}

Get a list of runs available in this Butler by querying for `raw` data, and asking for the set of `run` values:

In [6]:
runList = butler.queryMetadata('raw', ['run'])
print(runList)

['7086']


As expected, this list is a single element long, containing the expected run number.

From the `visit` key in Butler, we can get a list of "full" exposures available in the Butler.  This key is constructed by the Butler during data ingest, and is used to organize data taken at the same time (the time is used to construct the key).  This is also going to return a list, but we will only consider the first element for the time being.

In [7]:
dataId = {'run': runList[0]}
visitList = butler.queryMetadata('raw', ['visit'], dataId=dataId)
print(visitList[0])

258333657


From this single visit, determine the single CCD images that comprise that "full exposure", using the `detectorName` and `detector` keys.  Get one of those single-device images, and pull the formal device name from the header.

In [8]:
dataId.update({'visit': visitList[0]})
imageList = butler.queryMetadata('raw', ['detectorName', 'detector'], dataId=dataId)
print(imageList[0])

('S00', 0)


This query returns a DM `Exposure` object, which contains a set of (value, mask, variance) images, a WCS, and associated other metadata.  It can be displayed, but I am skipping this step for now.  Either of `detectorName` and `detector` can be used to uniquely identify the device image data, but as both are available in the results from the previous query, I'm supplying both.

In [9]:
dataId.update({'detectorName': imageList[0][0], 'detector': imageList[0][1]})
image = butler.get('raw', dataId=dataId)
print(image)

The DM metadata contains the original image header, accessible with DM methods.  From a scan of the full list (the commented `print(image.getMetadata())`, I was able to identify that the `LSST_NUM` keyword contained values that match the CCD device names.

In [10]:
deviceName = image.getMetadata().getScalar('LSST_NUM')
print(deviceName)
#print(image.getMetadata())

E2V-CCD250-260


RHL suggests that the header isn't entirely trustworthy, and that perhaps we should be pulling detector serial numbers from the camera geometry (as defined by the `obs_lsstCam/policy/ts8/R00.yaml` file).  This file is manually generated.  The `Exposure` selected above has a `Detector` property that links it to this camera geometry file.  From that `Detector`, we can request the serial number and the name, which identifies its position in the camera.

In [11]:
detector = image.getDetector()
#help(detector)
detectorDeviceName = detector.getSerial()
print(detector.getSerial())
print(detector.getName())

E2V-CCD250-266-Dev
R00_S00


eTraveler does not know about this `detectorDeviceName` based on previous explorations.  I will continue with the `deviceName` selected previously, and leave this difference for the future.

From the `run` and `deviceName` obtained from Butler, query the eTraveler system to obtain test results and test files.  This returns the raft name used for this run, and from the results, we have a list of `deviceNames` along with the arrays containing the per-amplifier values for each test.  I have pulled two test results as an example.

I have also attempted to pull files from the eTraveler system, but the call fails with a `GetResultsNoDataException`.  I don't know if this fails only for this run, or if there are other issues with my query.  To avoid issues, I've simply surrounded the query with a `try`.

In [12]:
for run in runList:
    print(run)
    raft_list, data = g.get_tests(site_type="I&T-Raft", run=run)
    res = g.get_results(test_type="bright_columns", data=data, device=raft_list)
    print(raft_list)
    print(res.keys())
    print(res[deviceName])
    res2 = g.get_results(test_type="ptc_gain", data=data, device=raft_list)
    print(res2[deviceName])
    try:
        testFiles = gf.get_files(run=run)
    except:
        testFiles = []
    print(testFiles)

7086
LCA-11021_RTM-007
odict_keys(['E2V-CCD250-260', 'E2V-CCD250-182', 'E2V-CCD250-175', 'E2V-CCD250-167', 'E2V-CCD250-195', 'E2V-CCD250-201', 'E2V-CCD250-222', 'E2V-CCD250-213', 'E2V-CCD250-177'])
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0.7101629376411438, 0.7143053412437439, 0.7206076383590698, 0.7239463329315186, 0.721765398979187, 0.73038649559021, 0.7361469864845276, 0.7402608394622803, 0.7090623378753662, 0.7078778743743896, 0.7105268836021423, 0.7073314189910889, 0.7138391733169556, 0.7099373936653137, 0.7040491104125977, 0.688077449798584]
[]


## This is quite a bit of information.  Can I do something useful with it?

As an example, is it possible to construct a R00.yaml based entirely on eTraveler data?

Explicitly set the run here, to ensure this section completes independently of the previous queries.  From that run, pull the test results that are used in the yaml file.

In [13]:
run = 7086  # Explicitly choosing the same as above.
raftList, data = g.get_tests(site_type="I&T-Raft", run=run)
gainRes = g.get_results(test_type="gain", data=data, device=raftList)
readNoiseRes = g.get_results(test_type="read_noise", data=data, device=raftList)

The raft information contains the CCD type, and contains the serial number for this raft configuration (?? I do not actually know what defines a raft serial number).

In [14]:
raftInfo = eR.raftContents(raftName=raftList, run=run)
raftType = eR.raft_type(raft=raftList)

print("R00 :")
print("  detectorType : %s" % (raftType))
print("  raftSerial : %s" % (raftList))
print("  ccdSerials :")

R00 :
  detectorType : e2v
  raftSerial : LCA-11021_RTM-007
  ccdSerials :


Iterate over the detectors of the raft, and print the mapping between the detector location on the raft, and the detector serial number.

In [15]:
# print(raftInfo) # (deviceName, deviceLocation, REB)
for ccd in raftInfo:
    print("    %s : %s" % (ccd[1], ccd[0]))

    S00 : E2V-CCD250-260
    S01 : E2V-CCD250-182
    S02 : E2V-CCD250-175
    S10 : E2V-CCD250-167
    S11 : E2V-CCD250-195
    S12 : E2V-CCD250-201
    S20 : E2V-CCD250-222
    S21 : E2V-CCD250-213
    S22 : E2V-CCD250-177


For each detector in the raft, we would like to know the gain and read noise for each amplifier.  I could not identify a location of the `C00` amplifier names in eTraveler, but I do not know if that means that information does not exist.  Instead, I've assumed that a simple iterator will provide the correct amplifier identifier.

In [16]:
print("")
print("  amplifiers :")
for ccd in raftInfo:
    print("    %s :" % (ccd[1]))
    for idx, (gain, rn) in enumerate(zip(gainRes[ccd[0]], readNoiseRes[ccd[0]])):
        print("      C%02o : { gain : %f, readNoise : %f}" % (idx, 1/gain, rn)) # this may not need to be 1/gain


  amplifiers :
    S00 :
      C00 : { gain : 1.425768, readNoise : 5.196356}
      C01 : { gain : 1.416527, readNoise : 4.981935}
      C02 : { gain : 1.397792, readNoise : 4.890378}
      C03 : { gain : 1.396435, readNoise : 5.003916}
      C04 : { gain : 1.396115, readNoise : 4.983557}
      C05 : { gain : 1.382393, readNoise : 5.162814}
      C06 : { gain : 1.366971, readNoise : 5.184981}
      C07 : { gain : 1.367701, readNoise : 5.297394}
      C10 : { gain : 1.416200, readNoise : 5.458410}
      C11 : { gain : 1.426139, readNoise : 5.273975}
      C12 : { gain : 1.416797, readNoise : 5.307252}
      C13 : { gain : 1.424342, readNoise : 5.134071}
      C14 : { gain : 1.408973, readNoise : 5.678471}
      C15 : { gain : 1.417583, readNoise : 5.734850}
      C16 : { gain : 1.428731, readNoise : 5.803617}
      C17 : { gain : 1.461123, readNoise : 5.937781}
    S01 :
      C00 : { gain : 1.381113, readNoise : 4.864641}
      C01 : { gain : 1.390683, readNoise : 4.865966}
      C02 

Put it all together to return the R00.yaml contents in a single cell output:

In [17]:
run = 7086  # Explicitly choosing the same as above.
raftList, data = g.get_tests(site_type="I&T-Raft", run=run)
gainRes = g.get_results(test_type="gain", data=data, device=raftList)
readNoiseRes = g.get_results(test_type="read_noise", data=data, device=raftList)

raftInfo = eR.raftContents(raftName=raftList, run=run)
raftType = eR.raft_type(raft=raftList)

print("R00 :")
print("  detectorType : %s" % (raftType))
print("  raftSerial : %s" % (raftList))
print("  ccdSerials :")

# print(raftInfo) # (deviceName, deviceLocation, deviceTravelerId???)
for ccd in raftInfo:
    print("    %s : %s" % (ccd[1], ccd[0]))
    
print("")
print("  amplifiers :")
for ccd in raftInfo:
    print("    %s :" % (ccd[1]))
    for idx, (gain, rn) in enumerate(zip(gainRes[ccd[0]], readNoiseRes[ccd[0]])):
        print("      C%02o : { gain : %f, readNoise : %f}" % (idx, 1/gain, rn))  # this may not need to be 1/gain

R00 :
  detectorType : e2v
  raftSerial : LCA-11021_RTM-007
  ccdSerials :
    S00 : E2V-CCD250-260
    S01 : E2V-CCD250-182
    S02 : E2V-CCD250-175
    S10 : E2V-CCD250-167
    S11 : E2V-CCD250-195
    S12 : E2V-CCD250-201
    S20 : E2V-CCD250-222
    S21 : E2V-CCD250-213
    S22 : E2V-CCD250-177

  amplifiers :
    S00 :
      C00 : { gain : 1.425768, readNoise : 5.196356}
      C01 : { gain : 1.416527, readNoise : 4.981935}
      C02 : { gain : 1.397792, readNoise : 4.890378}
      C03 : { gain : 1.396435, readNoise : 5.003916}
      C04 : { gain : 1.396115, readNoise : 4.983557}
      C05 : { gain : 1.382393, readNoise : 5.162814}
      C06 : { gain : 1.366971, readNoise : 5.184981}
      C07 : { gain : 1.367701, readNoise : 5.297394}
      C10 : { gain : 1.416200, readNoise : 5.458410}
      C11 : { gain : 1.426139, readNoise : 5.273975}
      C12 : { gain : 1.416797, readNoise : 5.307252}
      C13 : { gain : 1.424342, readNoise : 5.134071}
      C14 : { gain : 1.408973, readNoi